In [ ]:
import numpy as np
import matplotlib.pyplot as plt
Loss_train_adam = []
class MultiClassMLP:
    def __init__(self, input_dim, hidden_dims, output_dim, activation='relu', random_seed=42):
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.output_dim = output_dim
        self.layers = []   # weights and biases for each layer
        self.num_layers = len(hidden_dims) + 1
        parameters = {}

        # Initialize weights and biases for each layer
        np.random.seed(random_seed)
        layer_dims = [input_dim] + hidden_dims + [output_dim]
        self.activations = [
            self._sigmoid if activation == 'sigmoid' else self._tanh if activation == 'tanh' else self._relu for i in
            range(self.num_layers)]
        for i in range(self.num_layers):
            fan_in = layer_dims[i]
            fan_out = layer_dims[i + 1]
            W = np.random.randn(fan_in, fan_out) / np.sqrt(fan_in)
            b = np.zeros((1, fan_out))
            
            self.layers.append({'W': W, 'b': b})
            
    def find_range_weight():
        max = 0
        min = np.inf
        for i in range(self.num_layers):
            if max < np.max(self.layers[i]['W']):
                max = np.max(self.layers[i]['W'])
            if min > np.min(self.layers[i]['W']):
                min = np.min(self.layers[i]['W'])
        return max, min
    

    def update_parameters(self):
        self.parameters = {}
        for counter, i in enumerate(self.layers):
            self.parameters.update({"W" + str(counter + 1): i["W"], "b" + str(counter + 1): i["b"]})

    def update_layers(self):
        for i in range(self.num_layers):
            self.layers[i]['W'] = self.parameters["W" + str(i + 1)]
            self.layers[i]['b'] = self.parameters["b" + str(i + 1)]

    def _softmax(self, X):
        exps = np.exp(X - np.max(X, axis=1, keepdims=True))
        return exps / np.sum(exps, axis=1, keepdims=True)


    def _tanh(self, X):
        return np.tanh(X)

    def _sigmoid(self, X):
        return 1 / (1 + np.exp(-X))

    def _relu(self, X):
        return np.maximum(0, X)

    def _forward(self, X):
        activations = [X]
        for i in range(self.num_layers):
            Z = np.dot(activations[-1], self.layers[i]['W']) + self.layers[i]['b']
            A = self.activations[i](Z)
            activations.append(A)
        probs = self._softmax(activations[-1])  # Using softmax activation function for output layer
        return activations, probs

    def _backward(self, X, y, activations, probs, learning_rate):
        grads = {}
        dL_dO = probs - y
        for i in reversed(range(self.num_layers)):
            if self.activations[i] == self._sigmoid:
                dA = dL_dO * activations[i + 1] * (1 - activations[i + 1])
            elif self.activations[i] == self._tanh:
                dA = dL_dO * (1 - activations[i + 1] ** 2)  # Derivative of tanh activation function
            elif self.activations[i] == self._relu:
                dA = dL_dO * np.where(activations[i + 1] > 0, 1, 0)  # Derivative of ReLU activation function
            else:
                raise ValueError("Invalid activation function")

            dZ = np.dot(dA, self.layers[i]['W'].T)
            dW = np.dot(activations[i].T, dA)
            db = np.sum(dA, axis=0, keepdims=True)
            grads.update({"dW" + str(i + 1): dW, "db" + str(i + 1): db})
            # self.layers[i]['W'] -= learning_rate * dW
            # self.layers[i]['b'] -= learning_rate * db

            dL_dO = dZ
        return grads

    def initialize_adam(self, parameters):
        L = len(parameters) // 2
        v = {}
        s = {}
        for l in range(L):
            v["dW" + str(l + 1)] = np.zeros(
                (parameters["W" + str(l + 1)].shape[0], parameters["W" + str(l + 1)].shape[1]))
            v["db" + str(l + 1)] = np.zeros(
                (parameters["b" + str(l + 1)].shape[0], parameters["b" + str(l + 1)].shape[1]))
            s["dW" + str(l + 1)] = np.zeros(
                (parameters["W" + str(l + 1)].shape[0], parameters["W" + str(l + 1)].shape[1]))
            s["db" + str(l + 1)] = np.zeros(
                (parameters["b" + str(l + 1)].shape[0], parameters["b" + str(l + 1)].shape[1]))
        return v, s

    def update_parameters_with_adam(self, parameters, grads, v, s, t, learning_rate=0.01, beta1=0.9, beta2=0.999,
                                    epsilon=1e-8):
        """
        Update parameters using Adam

        Arguments:
        parameters -- python dictionary containing your parameters:
                        parameters['W' + str(l)] = Wl
                        parameters['b' + str(l)] = bl
        grads -- python dictionary containing your gradients for each parameters:
                        grads['dW' + str(l)] = dWl
                        grads['db' + str(l)] = dbl
        v -- Adam variable, moving average of the first gradient, python dictionary
        s -- Adam variable, moving average of the squared gradient, python dictionary
        learning_rate -- the learning rate, scalar.
        beta1 -- Exponential decay hyperparameter for the first moment estimates
        beta2 -- Exponential decay hyperparameter for the second moment estimates
        epsilon -- hyperparameter preventing division by zero in Adam updates

        Returns:
        parameters -- python dictionary containing your updated parameters
        v -- Adam variable, moving average of the first gradient, python dictionary
        s -- Adam variable, moving average of the squared gradient, python dictionary
        """

        L = len(parameters) // 2  # number of layers in the neural networks
        v_corrected = {}  # Initializing first moment estimate, python dictionary
        s_corrected = {}  # Initializing second moment estimate, python dictionary

        # Perform Adam update on all parameters
        for l in range(L):
            # Moving average of the gradients. Inputs: "v, grads, beta1". Output: "v".
            ### START CODE HERE ### (approx. 2 lines)
            v["dW" + str(l + 1)] = beta1 * v["dW" + str(l + 1)] + (1 - beta1) * grads['dW' + str(l + 1)]
            v["db" + str(l + 1)] = beta1 * v["db" + str(l + 1)] + (1 - beta1) * grads['db' + str(l + 1)]
            ### END CODE HERE ###

            # Compute bias-corrected first moment estimate. Inputs: "v, beta1, t". Output: "v_corrected".
            ### START CODE HERE ### (approx. 2 lines)
            v_corrected["dW" + str(l + 1)] = v["dW" + str(l + 1)] / (1 - beta1 ** t)
            v_corrected["db" + str(l + 1)] = v["db" + str(l + 1)] / (1 - beta1 ** t)
            ### END CODE HERE ###

            # Moving average of the squared gradients. Inputs: "s, grads, beta2". Output: "s".
            ### START CODE HERE ### (approx. 2 lines)
            s["dW" + str(l + 1)] = beta2 * s["dW" + str(l + 1)] + (1 - beta2) * np.square(grads['dW' + str(l + 1)])
            s["db" + str(l + 1)] = beta2 * s["db" + str(l + 1)] + (1 - beta2) * np.square(grads['db' + str(l + 1)])
            ### END CODE HERE ###

            # Compute bias-corrected second raw moment estimate. Inputs: "s, beta2, t". Output: "s_corrected".
            ### START CODE HERE ### (approx. 2 lines)
            s_corrected["dW" + str(l + 1)] = s["dW" + str(l + 1)] / (1 - beta2 ** t)
            s_corrected["db" + str(l + 1)] = s["db" + str(l + 1)] / (1 - beta2 ** t)
            ### END CODE HERE ###

            # Update parameters. Inputs: "parameters, learning_rate, v_corrected, s_corrected, epsilon". Output: "parameters".
            ### START CODE HERE ### (approx. 2 lines)
            parameters["W" + str(l + 1)] = parameters["W" + str(l + 1)] - learning_rate * v_corrected[
                "dW" + str(l + 1)] / (np.sqrt(s_corrected["dW" + str(l + 1)]) + epsilon)
            parameters["b" + str(l + 1)] = parameters["b" + str(l + 1)] - learning_rate * v_corrected[
                "db" + str(l + 1)] / (np.sqrt(s_corrected["db" + str(l + 1)]) + epsilon)
            ### END CODE HERE ###

        return parameters, v, s

    def train(self, X_train, y_train, learning_rate=0.0007, mini_batch_size=64, beta=0.9,
              beta1=0.9, beta2=0.999, epsilon=1e-8, num_epochs=10000, print_cost=True):
        Y_train = np.eye(self.output_dim)[y_train]
        t = 0
        self.update_parameters()
        v, s = self.initialize_adam(self.parameters)
        for epoch in range(num_epochs):
            activations, probs = self._forward(X_train)
            grads = self._backward(X_train, Y_train, activations, probs, learning_rate)
            self.update_parameters()
            t = t + 1
            self.parameters, v, s = self.update_parameters_with_adam(self.parameters, grads, v, s,
                                                                t, learning_rate, beta1, beta2, epsilon)
            self.update_layers()
            loss = -np.sum(Y_train * np.log(probs)) / X_train.shape[0]
            if epoch % 10 == 0:
                Loss_train_adam.append(loss)
                print(f"Epoch {epoch}: Loss={loss:.4f}")

    def predict_proba(self, X):
        _, probs = self._forward(X)
        return probs

    def predict(self, X):
        probs = self.predict_proba(X)
        return np.argmax(probs, axis=1)


    


In [ ]:
import matplotlib.pyplot as plt
# torch is just for the feature extractor and the dataset (NOT FOR IMPLEMENTING NEURAL NETWORKS!)
import torch
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.models import resnet34
import torch.nn as nn
# sklearn is just for evaluation (NOT FOR IMPLEMENTING NEURAL NETWORKS!)
from sklearn.metrics import confusion_matrix, f1_score

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
feature_extractor = resnet34(pretrained=True)
input_dim = feature_extractor.fc.in_features
for param in feature_extractor.parameters():
  param.requires_grad = False

feature_extractor.fc = nn.Identity()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform)
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

In [ ]:
embeddings = []
labels = []
i = 0
for x, y in train_loader:
  if i == 50:
    break
  i += 1
  print(i)
  embeddings += feature_extractor(x)
  labels += y

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [ ]:
 len(next(iter(train_loader))[0])

64

In [ ]:
len(embeddings)

3200

In [ ]:
labels

[tensor(9),
 tensor(4),
 tensor(0),
 tensor(9),
 tensor(5),
 tensor(8),
 tensor(4),
 tensor(2),
 tensor(2),
 tensor(8),
 tensor(5),
 tensor(3),
 tensor(1),
 tensor(9),
 tensor(5),
 tensor(8),
 tensor(9),
 tensor(0),
 tensor(1),
 tensor(6),
 tensor(4),
 tensor(3),
 tensor(4),
 tensor(8),
 tensor(1),
 tensor(5),
 tensor(7),
 tensor(4),
 tensor(4),
 tensor(0),
 tensor(1),
 tensor(9),
 tensor(7),
 tensor(1),
 tensor(3),
 tensor(7),
 tensor(0),
 tensor(2),
 tensor(5),
 tensor(6),
 tensor(4),
 tensor(4),
 tensor(6),
 tensor(5),
 tensor(6),
 tensor(1),
 tensor(5),
 tensor(6),
 tensor(6),
 tensor(7),
 tensor(3),
 tensor(8),
 tensor(2),
 tensor(5),
 tensor(5),
 tensor(3),
 tensor(5),
 tensor(8),
 tensor(9),
 tensor(7),
 tensor(9),
 tensor(9),
 tensor(4),
 tensor(8),
 tensor(3),
 tensor(1),
 tensor(1),
 tensor(4),
 tensor(6),
 tensor(0),
 tensor(4),
 tensor(8),
 tensor(4),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(5),
 tensor(6),
 tensor(3),
 tensor(0),
 tensor(5),
 tensor(3),
 tensor(3),
 ten

In [ ]:
# data_tensor = torch.cat(embeddings, dim=0)

# Convert the tensor to a NumPy array
X = np.array(embeddings)
# data_tensor = torch.cat(labels, dim=0)
for i in range(len(X)):
  X[i] = np.array(X[i])
# Convert the tensor to a NumPy array
Y = np.array(labels)

<ipython-input-29-e56cbe7bc52a>:4: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  X = np.array(embeddings)
<ipython-input-29-e56cbe7bc52a>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(embeddings)


In [ ]:
result = []
for i in X:
  result.append(i)
result = np.array(result)
result.shape

(3200, 512)

In [ ]:
Y.flatten().shape

(3200,)

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1, max_iter=1000, hidden_layer_sizes=[20]).fit(result, Y)
clf.score(result, Y)

0.999375

In [ ]:
def chart():
    # Generate x-axis values using range() function
    x = range(len(Loss_train_adam))

    # Create line plot
    plt.plot(x, Loss_train_adam)

    # Add labels and title
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss vs. Epochs for Adam Optimizer')

    # Show the plot
    plt.show()

In [ ]:
mlp = MultiClassMLP(input_dim=512, hidden_dims=[100], output_dim=10, activation="tanh")
mlp.train(result, Y, num_epochs=1000)
chart()


Epoch 0: Loss=2.4393
Epoch 10: Loss=2.0064
Epoch 20: Loss=1.7976
Epoch 30: Loss=1.6791
Epoch 40: Loss=1.5942
Epoch 50: Loss=1.5248
Epoch 60: Loss=1.4646
Epoch 70: Loss=1.4112
Epoch 80: Loss=1.3632
Epoch 90: Loss=1.3196
Epoch 100: Loss=1.2796
Epoch 110: Loss=1.2428
Epoch 120: Loss=1.2094
Epoch 130: Loss=1.1791
Epoch 140: Loss=1.1516
Epoch 150: Loss=1.1266
Epoch 160: Loss=1.1037
Epoch 170: Loss=1.0827
Epoch 180: Loss=1.0634
Epoch 190: Loss=1.0458
Epoch 200: Loss=1.0298
Epoch 210: Loss=1.0151
Epoch 220: Loss=1.0015
Epoch 230: Loss=0.9893
Epoch 240: Loss=0.9783
Epoch 250: Loss=0.9683
Epoch 260: Loss=0.9592
Epoch 270: Loss=0.9511
Epoch 280: Loss=0.9436
Epoch 290: Loss=0.9368
Epoch 300: Loss=0.9304
Epoch 310: Loss=0.9245
Epoch 320: Loss=0.9193
Epoch 330: Loss=0.9145
Epoch 340: Loss=0.9102
Epoch 350: Loss=0.9061
Epoch 360: Loss=0.9024
Epoch 370: Loss=0.8989
Epoch 380: Loss=0.8956
Epoch 390: Loss=0.8927
Epoch 400: Loss=0.8900
Epoch 410: Loss=0.8876
Epoch 420: Loss=0.8853
Epoch 430: Loss=0.8831

In [ ]:
counter = 0
for i, t in enumerate(result):
  if mlp.predict(t) == Y[i]:
    counter += 1
print(counter / len(result))

0.9646875


In [ ]:
print(mlp.find_range_weight())

In [ ]:
X_train = np.random.randn(100, 10)
y_train = np.random.randint(0, 5, size=100)
# Y_train = np.eye(5)[y_train]

# Initialize and train the MLP
mlp2 = MultiClassMLP(input_dim=10, hidden_dims=[20], output_dim=5, activation="sigmoid")
mlp2.train(X_train, y_train, learning_rate=0.1, num_epochs=1000)

Epoch 0: Loss=1.5996
Epoch 10: Loss=1.5479
Epoch 20: Loss=1.5098
Epoch 30: Loss=1.4817
Epoch 40: Loss=1.4593
Epoch 50: Loss=1.4382
Epoch 60: Loss=1.4175
Epoch 70: Loss=1.3972
Epoch 80: Loss=1.3778
Epoch 90: Loss=1.3594
Epoch 100: Loss=1.3424
Epoch 110: Loss=1.3268
Epoch 120: Loss=1.3122
Epoch 130: Loss=1.2984
Epoch 140: Loss=1.2854
Epoch 150: Loss=1.2729
Epoch 160: Loss=1.2605
Epoch 170: Loss=1.2485
Epoch 180: Loss=1.2379
Epoch 190: Loss=1.2283
Epoch 200: Loss=1.2195
Epoch 210: Loss=1.2114
Epoch 220: Loss=1.2039
Epoch 230: Loss=1.1970
Epoch 240: Loss=1.1905
Epoch 250: Loss=1.1842
Epoch 260: Loss=1.1781
Epoch 270: Loss=1.1726
Epoch 280: Loss=1.1675
Epoch 290: Loss=1.1631
Epoch 300: Loss=1.1593
Epoch 310: Loss=1.1560
Epoch 320: Loss=1.1531
Epoch 330: Loss=1.1504
Epoch 340: Loss=1.1480
Epoch 350: Loss=1.1458
Epoch 360: Loss=1.1438
Epoch 370: Loss=1.1418
Epoch 380: Loss=1.1399
Epoch 390: Loss=1.1381
Epoch 400: Loss=1.1364
Epoch 410: Loss=1.1347
Epoch 420: Loss=1.1332
Epoch 430: Loss=1.1318